In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 80)

In [9]:
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

files_id = {
    'housing_data':"1VEpP7kLJjlLR9MyTgu2FyFnOJArh6U2b", #iteration6 data
    'test_housing_data':"1CMsAWhWKWBjI6DDEHtcYmRVZRazfE9bo", #test data for housing
    'ids_com':"10gwiL49calkj-xbx-3rQEK4H2zoJcU11" #ID for the commiting of the project

}


housing_data = pd.read_csv(gd_path(files_id['housing_data']), sep=",")
test_housing_data = pd.read_csv(gd_path(files_id['test_housing_data']), sep=",")
ids_com = pd.read_csv(gd_path(files_id['ids_com']), sep=",")
df = housing_data

# Pre-Processing Pipe

## Ordinary columns

This part was done by hands. Many Bothans died to bring us this information.

In [10]:
from sklearn.model_selection import train_test_split

# X and y creation
y = df.pop("Expensive")

# Feature Engineering
X = df

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
# building the pipeline
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

#  numerical pipeline
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"))


# categorical pipeline
# defining ordinal & onehot columns

ordinal_cols = X_cat.columns.get_indexer(['LandContour', 'LandSlope',
                     'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                     'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                     'HeatingQC', 'KitchenQual', 'Functional',
                     'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                     'PoolQC']) 

onehot_cols = X_cat.columns.get_indexer(['MSZoning', 'Condition1', 'Heating', 'Street', 'CentralAir', 
                                         'Foundation', 'Alley', 'LotShape', 'Utilities', 'LotConfig', 
                                         'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 
                                         'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                                         'Electrical', 'GarageType', 'PavedDrive', 'Fence', 'MiscFeature',
                                         'SaleType', 'SaleCondition'])



# defining the categorical encoder
# we manually establish the order of the categories for our ordinal feature (Cabin), including "N_A"
LandContour = ['Lvl', 'Bnk', 'HLS', 'Low']
LandSlope = ['Gtl', 'Mod', 'Sev']
ExterQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
ExterCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
BsmtQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtExposure = ['Gd', 'Av', 'Mn', 'No', 'N_A']
BsmtFinType1 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
BsmtFinType2 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
HeatingQC = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
KitchenQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
Functional = ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal']
FireplaceQu = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageFinish = ['Fin', 'RFn', 'Unf', 'N_A']
GarageQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
PoolQC = ['Ex', 'Gd', 'TA', 'Fa', 'N_A']


categorical_encoder = ColumnTransformer(
    transformers=[
        ("cat_ordinal", OrdinalEncoder(categories=[LandContour,LandSlope,
                                                    ExterQual,ExterCond, BsmtQual, BsmtCond,
                                                    BsmtExposure, BsmtFinType1, BsmtFinType2,
                                                    HeatingQC, KitchenQual, Functional,
                                                    FireplaceQu, GarageFinish, GarageQual, GarageCond,
                                                    PoolQC]), ordinal_cols),

        ("cat_onehot", OneHotEncoder(handle_unknown="ignore"), onehot_cols)
    ]
)


categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    categorical_encoder 
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        ("cat_pipe", categoric_pipe, X_cat.columns),
    ]
)

# Modelling Pipe - 1

## Decisiontree - Creation

In [272]:
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              DecisionTreeClassifier())
'''
param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}'''

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    'decisiontreeclassifier__max_depth': range(2, 20),
    'decisiontreeclassifier__min_samples_leaf': range(3, 15, 2),
    'decisiontreeclassifier__min_samples_split': range(3, 40, 5),
    'decisiontreeclassifier__criterion':['gini', 'entropy']
    }

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"dtree" : search.best_score_}

scores

Fitting 5 folds for each of 3456 candidates, totalling 17280 fits


KeyboardInterrupt: 

## Decisiontree - Analysis

In [12]:
# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")

print("Confusion Matrix (Training):\n", confusion_matrix_train,"\n")
print("Confusion Matrix (Testing):\n", confusion_matrix_test,"\n")

print("F-Score:\n")
print("Training Score:", round(f1_score(y_train,y_train_pred),3))
print("Testing Score:", round(f1_score(y_test,y_test_pred),3),"\n")

print("Kappa-Score:\n")
print("Training Score:", round(cohen_kappa_score(y_train,y_train_pred),3))
print("Testing Score:", round(cohen_kappa_score(y_test,y_test_pred),3))

NameError: name 'search' is not defined

# Modeling Pipe - 2


In [ ]:
# Modeling Pipe - 2

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


full_pipeline = make_pipeline(preprocessor,
                              KNeighborsClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "kneighborsclassifier__n_neighbors": range(3, 20, 2)
}

search2 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=10,
                      verbose=1)

search2.fit(X_train, y_train)

scores2 = {"knn" : search.best_score_}

scores2


Fitting 10 folds for each of 18 candidates, totalling 180 fits


{'knn': 0.9280694031766993}

## Decisiontree - Analysis KNN

### Confusion Matrix

In [ ]:
# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")

print("Confusion Matrix (Training):\n", confusion_matrix_train,"\n")
print("Confusion Matrix (Testing):\n", confusion_matrix_test,"\n")

print("F-Score:\n")
print("Training Score:", round(f1_score(y_train,y_train_pred),3))
print("Testing Score:", round(f1_score(y_test,y_test_pred),3),"\n")

print("Kappa-Score:\n")
print("Training Score:", round(cohen_kappa_score(y_train,y_train_pred),3))
print("Testing Score:", round(cohen_kappa_score(y_test,y_test_pred),3))

Training Accuracy: 0.9700342465753424
Testing Accuracy: 0.928082191780822
Confusion Matrix (Training):
 [[980  13]
 [ 22 153]]
Confusion Matrix (Testing):
 [[240  10]
 [ 11  31]]


# Modelling Pipe - 3

## RandomForest- Creation

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              RandomForestClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy": ["median", "mean"],
    "randomforestclassifier__n_estimators": range(10, 101, 10),
    "randomforestclassifier__max_depth": range(4, 16, 2),
    "randomforestclassifier__min_samples_split": range(2, 11, 2),
    "randomforestclassifier__min_samples_leaf": range(1, 6, 2),
    "randomforestclassifier__max_features": ["sqrt", "log2", None],
}

# need to be rearange 
search4 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search4.fit(X_train, y_train)

scores4 = {"rf" : search4.best_score_}
print("Best parameters: ", search4.best_params_)
scores4

Fitting 5 folds for each of 5400 candidates, totalling 27000 fits
Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'randomforestclassifier__max_depth': 14, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 4, 'randomforestclassifier__n_estimators': 50}


{'rf': 0.950331976083049}

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              RandomForestClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy": ["median"],
    "randomforestclassifier__n_estimators": [50],
    "randomforestclassifier__max_depth": [14],
    "randomforestclassifier__min_samples_split": [4],
    "randomforestclassifier__min_samples_leaf": [1],
    "randomforestclassifier__max_features": ["sqrt"],
}

# need to be rearange 
search4 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search4.fit(X_train, y_train)

scores4 = {"rf" : search4.best_score_}
print("Best parameters: ", search4.best_params_)
scores4

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'randomforestclassifier__max_depth': 14, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__min_samples_split': 4, 'randomforestclassifier__n_estimators': 50}


{'rf': 0.9426286636587067}

Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'median', 
                    'randomforestclassifier__max_depth': 14,
                    'randomforestclassifier__max_features': 'sqrt', 
                    'randomforestclassifier__min_samples_leaf': 1, 
                    'randomforestclassifier__min_samples_split': 4, 
                    'randomforestclassifier__n_estimators': 50}
Fitting 5 folds for each of 5400 candidates, totalling 27000 fits

63, 20.6 S


## RandomForest - Analysis

In [15]:
# Get predictions for the training and testing datasets
y_train_pred = search4.predict(X_train)
y_test_pred = search4.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")

print("Confusion Matrix (Training):\n", confusion_matrix_train,"\n")
print("Confusion Matrix (Testing):\n", confusion_matrix_test,"\n")

print("F-Score:\n")
print("Training Score:", round(f1_score(y_train,y_train_pred),3))
print("Testing Score:", round(f1_score(y_test,y_test_pred),3),"\n")

print("Kappa-Score:\n")
print("Training Score:", round(cohen_kappa_score(y_train,y_train_pred),3))
print("Testing Score:", round(cohen_kappa_score(y_test,y_test_pred),3))

Training Accuracy: 1.0
Testing Accuracy: 0.966 

Confusion Matrix (Training):
 [[993   0]
 [  0 175]] 

Confusion Matrix (Testing):
 [[247   3]
 [  7  35]] 

F-Score:

Training Score: 1.0
Testing Score: 0.875 

Kappa-Score:

Training Score: 1.0
Testing Score: 0.855


## Downloand

In [16]:
X_sumbmition = housing_data[0:1459]
ids_com['Expensive'] = search4.predict(X_sumbmition) # only cange the piplene! 
ids_com.to_csv(r'submission_test.csv', index=False) #only cahnge ist 